In [ ]:
import json

import pandas as pd

# Load

In [ ]:
beneficiaries = pd.read_csv('data/esif_2014_2020_beneficiaries_lat_lon.csv')

In [ ]:
beneficiaries.head()

In [ ]:
beneficiaries = beneficiaries.rename(columns={
    'Coût total du projet £m': 'project_cost', 
    'Investissement FEDER/FSE £m': 'eu_investment',
    '% du projet cofinancé par l’UE': 'prop_eu_financed',
    'latitude': 'lat', 
    'longitude': 'lng',
    'Bénéficiaire': 'beneficiary',
    'Nom du projet': 'project',
    'Zone de partenariat économique local': 'economic_zone',
    'Résumé du projet': 'summary',
    'Date de commencement': 'start_date'
})

In [ ]:
beneficiaries['start_date'] = pd.to_datetime(beneficiaries.start_date, format='%Y-%m-%d')

In [ ]:
beneficiaries[['project_cost', 'eu_investment']] = beneficiaries[['project_cost', 'eu_investment']].apply(
    lambda s: s.str.strip().str.replace(',', '').str.replace(r'(\.)[\d|\.]{3,}\b', '')
).astype(float)

In [ ]:
beneficiaries['project'] = beneficiaries.project.str.replace('\n', '').str.replace("''", '')

In [ ]:
beneficiaries[['beneficiary', 'project', 'summary', 'start_date', 'project_cost', 'eu_investment', 'prop_eu_financed', 'lat', 'lng']].head()

# Save 

In [ ]:
beneficiaries.to_pickle('data/beneficiaries.pkl')

In [ ]:
def make_beneficiaries_data_geo_json(beneficiaries):
    def make_feature(row):
        properties = {
            property: row[property]
            for property in ['beneficiary', 'project', 'project_cost', 'eu_investment']
        }
        return {
            'type': 'Feature',
            'geometry': {
                "type": "Point",
                "coordinates": [row['lng'], row['lat']]
            },
            'properties': properties
        }
    features = list(beneficiaries.apply(make_feature, axis=1))
    return { 'type': 'FeatureCollection', 'features': features }
with open('data/beneficiaries.geo.json', 'w') as file:
    json.dump(make_beneficiaries_data_geo_json(beneficiaries), file, sort_keys=True)